# Causal Abstraction


In [1]:
import numpy as np
import os
import pandas as pd
from scipy.stats import gaussian_kde
from conditions import MID_POINT_RAMP, scenarios, groundtruth_positions, trial_configs
from step1_physics_simulations import run_single_trial, run_all_combinations, inspect_symmetry
from step2_bayesian_inference import (
    fit_kde_models, compute_combined_choice_scores,
    define_prior, create_probability_table, calculate_posterior, 
    posterior_weighted_score, softmax_choice_probabilities
)
from step3_optimize_parameters import (
    load_participant_data, calculate_log_likelihood, optimize_parameters, 
    load_participant_data
)
# Display options
from IPython.display import display
pd.set_option('display.max_rows', 48)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


# 1. Physical Simulations


## 1.1 Scenarios
Four types of simulation will be ran: forward ramp - downward motion, forward ramp - upward motion, backward ramp - downward motion, backward ramp - upward mption. This section shows the mapping between trial - model type and types of simulations.

In [2]:
# Scenarios that will be simulated
scenario_df = pd.DataFrame([
    {'sim_type': sim_type, 'trial_name': trial['name'], 'model_type': trial['model_type']}
    for sim_type, trial in scenarios
])

display(scenario_df)

,sim_type,trial_name,model_type
0,forward_ramp_down,trial_a_red_block,agent_physics
1,forward_ramp_up,trial_a_red_block,ramp
2,forward_ramp_down,trial_a_black_block,agent_physics
3,forward_ramp_up,trial_a_black_block,ramp
4,forward_ramp_down,trial_b_blue_ramp,agent_physics
5,forward_ramp_up,trial_b_blue_ramp,ramp
6,forward_ramp_down,trial_b_yellow_ramp,agent_physics
7,forward_ramp_up,trial_b_yellow_ramp,ramp


## 1.2 Simulation for One Parameter Combination
In this section, we generate physical simulations for a single parameter combination.  

- **Trial A**: Block noise is added. Upward motion simulates final positions for the ramp model. Downward motion simulates final positions for the physics and agent model.
- **Trial B**: Ramp noise is added.  Upward motion simulates final positions for the ramp model. Downward motion simulates final positions for the physics and agent model.
- **Trial C**: Final positions are computed using a symmetric calculation of positions in trial A.
- **Trial D**: Final positions are again computed using a symmetric calculation of positions in trial B.

The resulting simulated data will be used in later sections for Bayesian inference and model fitting.

In [3]:
BLOCK_NOISE_SD = 0.05
RAMP_NOISE_SD = 0.5
N_TRIALS = 2000

# Combine all trial configs from both trial types
all_trial_configs = {**trial_configs["a"], **trial_configs["b"]}
all_results = []

for trial_name, trial_config in all_trial_configs.items():
    trial_config_with_name = {**trial_config, "name": trial_name}
    trial_data,failed_data = run_single_trial(trial_config_with_name, BLOCK_NOISE_SD, RAMP_NOISE_SD, N_TRIALS)
    all_results.extend(trial_data)

results_df = pd.DataFrame(all_results)
print(f"Generated {len(results_df)} total records")

output_file = "data/trial_results_demo.csv"
results_df.to_csv(output_file, index=False)
print(f"Results saved to: {output_file}")

print("\nFirst 10 rows:")
display(results_df.head(10))

Generated 48000 total records
Results saved to: data/trial_results_demo.csv

First 10 rows:


,trial,element,model,block_noise_sd,ramp_noise_sd,block_noise_down,ramp_noise_down,block_noise_up,ramp_noise_up,final_position
0,a,red_block,agent,0.05,0.5,0.116908,0.0,0.000000,0.0,967.880044
1,a,red_block,physics,0.05,0.5,0.116908,0.0,0.000000,0.0,967.880044
2,a,red_block,ramp,0.05,0.5,0.000000,0.0,0.019743,0.0,251.672330
3,a,red_block,agent,0.05,0.5,0.004735,0.0,0.000000,0.0,1081.121598
4,a,red_block,physics,0.05,0.5,0.004735,0.0,0.000000,0.0,1081.121598
5,a,red_block,ramp,0.05,0.5,0.000000,0.0,-0.046569,0.0,163.811647
6,a,red_block,agent,0.05,0.5,0.002043,0.0,0.000000,0.0,1085.559252
7,a,red_block,physics,0.05,0.5,0.002043,0.0,0.000000,0.0,1085.559252
8,a,red_block,ramp,0.05,0.5,0.000000,0.0,-0.023045,0.0,202.332060
9,a,red_block,agent,0.05,0.5,-0.058942,0.0,0.000000,0.0,1217.111019


## 1.3 Insights
This is for better understanding data generated in the above section.

In [4]:
# For each element, model, and trial, calculate the mean and std of the final position
summary_stats = results_df.groupby(['trial', 'element', 'model']).agg({
    'final_position': ['count', 'mean', 'std']
}).round(2)
summary_stats.columns = ['Count', 'Mean', 'Std_Dev']
display(summary_stats)

Count     Mean  Std_Dev
trial element     model                           
a     black_block agent     2000   969.68    35.92
                  physics   2000   969.68    35.92
                  ramp      2000   322.94    26.82
      red_block   agent     2000  1113.06    90.74
                  physics   2000  1113.06    90.74
                  ramp      2000   216.85    70.77
b     blue_ramp   agent     2000  1009.69    52.04
                  physics   2000  1009.69    52.04
                  ramp      2000   313.95    79.86
      yellow_ramp agent     2000   968.44    64.41
                  physics   2000   968.44    64.41
                  ramp      2000   345.06    82.24
c     black_block agent     2000   987.06    26.82
                  physics   2000   340.32    35.92
                  ramp      2000   987.06    26.82
      red_block   agent     2000  1093.15    70.77
                  physics   2000   196.94    90.74
                  ramp      2000  1093.15    70.77
d     blue_ramp   agent     2000   996.05    79.86
                  physics   2000   300.31    52.04
                  ramp      2000   996.05    79.86
      yellow_ramp agent     2000   964.94    82.24
                  physics   2000   341.56    64.41
                  ramp      2000   964.94    82.24

## 1.4 Symmetry Calculation
Positions for trials C and D are generated from A and B by flipping around the midpoint of the ramp. This is a helper function used in run_single_trial.

In [5]:
symmetry_result = inspect_symmetry(
    block_noise_sd=0.001,
    ramp_noise_sd=0.5,
    n_trials=1000,
)
symmetry_df, forward_down, forward_up, symmetry_analysis_df = symmetry_result



Symmetry Analysis:
Ramp midpoint: 655.0
Total records: 24000

red_block - Symmetry Check:
  (All values are averages of 3000 simulations)
  Physics Model (down motion):
    Trial A avg position: 1088.9 (distance from midpoint: 433.9)
    Trial C avg position: 221.1 (distance from midpoint: 433.9)
  Ramp Model (up motion):
    Trial A avg position: 231.0 (distance from midpoint: 424.0)
    Trial C avg position: 1079.0 (distance from midpoint: 424.0)

black_block - Symmetry Check:
  (All values are averages of 3000 simulations)
  Physics Model (down motion):
    Trial A avg position: 972.6 (distance from midpoint: 317.6)
    Trial C avg position: 337.4 (distance from midpoint: 317.6)
  Ramp Model (up motion):
    Trial A avg position: 327.1 (distance from midpoint: 327.9)
    Trial C avg position: 982.9 (distance from midpoint: 327.9)


## 1.5 Run multiple combinatoins
First, we simulate each unique block trial with different noise values, then we simulate each unique ramp trial with different noise values. Then, we combine the results.



In [6]:
# Run multiple parameter combinations 

# Block noise parameters
BLOCK_NOISE_SD_MIN = 0.05
BLOCK_NOISE_SD_MAX = 0.15
BLOCK_NOISE_SD_STEPS = 3

# Ramp noise parameters  
RAMP_NOISE_SD_MIN = 0.05
RAMP_NOISE_SD_MAX = 0.15
RAMP_NOISE_SD_STEPS = 3

# Simulation parameters
N_TRIALS = 100
SEED = 1

# Run all combinations with custom parameters
results_df = run_all_combinations(
    n_trials=N_TRIALS,
    seed=SEED,
    block_noise_sd_min=BLOCK_NOISE_SD_MIN,
    block_noise_sd_max=BLOCK_NOISE_SD_MAX,
    block_noise_sd_steps=BLOCK_NOISE_SD_STEPS,
    ramp_noise_sd_min=RAMP_NOISE_SD_MIN,
    ramp_noise_sd_max=RAMP_NOISE_SD_MAX,
    ramp_noise_sd_steps=RAMP_NOISE_SD_STEPS,
    save_results=False
)
print(results_df)

Running simulations using 5 parallel processes
This may take a few minutes...


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/la

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
Processing simulation results...
Combining results (files not saved)...
   block_noise_sd  ramp_noise_sd  total_failed_attempts  \
0            0.05           0.05                      0   
1            0.05           0.10                      1   
2            0.05           0.15                      7   
3            0.10           0.05                      0   
4            0.10           0.10                      1   
5        

# 2. Bayesian Inference

## 2.1 Kernal Density Estimation Implementation
We apply KDE to the simulations and compute a likelihood score for the four possible locations. Each trial has two elements (e.g., red and black blocks), we combine their scores at each location by multiplying the likelihoods of the two elements at the location.

In [7]:
# Uses phyiscal simulation data from section 1.2 to run KDE analysis
input_csv = "data/trial_results_demo.csv"   
output_csv = "data/kde_results_demo.csv"

bandwidth = 2

df = pd.read_csv (str(input_csv))
# Fitting for each trial(a/b/c/d), element (red_block/black_block/blue_ramp/yellow_ramp) and model(physics/agent/ramp) combination
kde_models = fit_kde_models(df, bandwidth)

choice_scores = compute_combined_choice_scores(kde_models)
choice_scores.to_csv(output_csv, index=False)
display(choice_scores)

display(kde_models)

Fitted KDE for trial_a_red_block - physics: 2000 points
Fitted KDE for trial_a_red_block - agent: 2000 points
Fitted KDE for trial_a_red_block - ramp: 2000 points
Fitted KDE for trial_a_black_block - physics: 2000 points
Fitted KDE for trial_a_black_block - agent: 2000 points
Fitted KDE for trial_a_black_block - ramp: 2000 points
Fitted KDE for trial_b_blue_ramp - physics: 2000 points
Fitted KDE for trial_b_blue_ramp - agent: 2000 points
Fitted KDE for trial_b_blue_ramp - ramp: 2000 points
Fitted KDE for trial_b_yellow_ramp - physics: 2000 points
Fitted KDE for trial_b_yellow_ramp - agent: 2000 points
Fitted KDE for trial_b_yellow_ramp - ramp: 2000 points
Fitted KDE for trial_c_red_block - physics: 2000 points
Fitted KDE for trial_c_red_block - agent: 2000 points
Fitted KDE for trial_c_red_block - ramp: 2000 points
Fitted KDE for trial_c_black_block - physics: 2000 points
Fitted KDE for trial_c_black_block - agent: 2000 points
Fitted KDE for trial_c_black_block - ramp: 2000 points
Fitt

,trial,model,choice_1,choice_2,choice_3,choice_4
0,trial_a,physics,1.915867e-25,1.591857e-29,2.560836e-06,9.962633e-06
1,trial_a,agent,1.915867e-25,1.591857e-29,2.560836e-06,9.962633e-06
2,trial_a,ramp,1.720614e-05,4.096619e-06,5.914388e-51,2.208205e-41
3,trial_b,physics,1.723698e-18,2.480783e-19,8.294345e-06,7.073259e-06
4,trial_b,agent,1.723698e-18,2.480783e-19,8.294345e-06,7.073259e-06
5,trial_b,ramp,3.831968e-06,4.239745e-06,1.102145e-12,1.276590e-12
6,trial_c,physics,9.962633e-06,2.560836e-06,1.591857e-29,1.915867e-25
7,trial_c,agent,2.208205e-41,5.914388e-51,4.096619e-06,1.720614e-05
8,trial_c,ramp,2.208205e-41,5.914388e-51,4.096619e-06,1.720614e-05
9,trial_d,physics,7.073259e-06,8.294345e-06,2.480783e-19,1.723698e-18


{'trial_a_red_block': {'physics': <scipy.stats._kde.gaussian_kde at 0x14b1c33b0>,
  'agent': <scipy.stats._kde.gaussian_kde at 0x14a5c9ac0>,
  'ramp': <scipy.stats._kde.gaussian_kde at 0x14a5c9190>},
 'trial_a_black_block': {'physics': <scipy.stats._kde.gaussian_kde at 0x14a5cb650>,
  'agent': <scipy.stats._kde.gaussian_kde at 0x14a5ca180>,
  'ramp': <scipy.stats._kde.gaussian_kde at 0x14a5c94c0>},
 'trial_b_blue_ramp': {'physics': <scipy.stats._kde.gaussian_kde at 0x14b1c1880>,
  'agent': <scipy.stats._kde.gaussian_kde at 0x14b1c1850>,
  'ramp': <scipy.stats._kde.gaussian_kde at 0x14b1c31d0>},
 'trial_b_yellow_ramp': {'physics': <scipy.stats._kde.gaussian_kde at 0x14b1c30e0>,
  'agent': <scipy.stats._kde.gaussian_kde at 0x14b1c3aa0>,
  'ramp': <scipy.stats._kde.gaussian_kde at 0x14b1c3a40>},
 'trial_c_red_block': {'physics': <scipy.stats._kde.gaussian_kde at 0x14b1c2930>,
  'agent': <scipy.stats._kde.gaussian_kde at 0x14b1c37a0>,
  'ramp': <scipy.stats._kde.gaussian_kde at 0x14b1c08f0

## 2.2 Define Priors 
We define a prior distribution over the three hypotheses about how the data could have been generated. 

Physics model: p, Agent model: (1-p) * q, Ramp model: (1-p) * (1-q)

In [8]:
p=0.9
q=0.5
prior = define_prior(p, q, show_details=True)

Prior probabilities:
  p(H_physics) = p = 0.900
  p(H_agent) = (1-p) * q = (1-0.9) * 0.5 = 0.050
  p(H_ramp) = (1-p) * (1-q) = (1-0.9) * (1-0.5) = 0.050


## 2.3 Create Probability Table
Creates a likelihood table that showcase how well a given hypothesis explains the observed data.

In [9]:
probability_table = create_probability_table(r=0.05)
display(probability_table)

,Block on left side of forward ramp,Block on right side of forward ramp,Block on left side of backward ramp,Block on right side of backward ramp
Hypothesis,,,,
Physics,0.05,1.00,1.00,0.05
Agent,0.05,1.00,0.05,1.00
Ramp,1.00,0.05,0.05,1.00


## 2.4 Calculate Posteriors
Computes the posterior belief p(hypothesis|data) for both the forward ramp condition and the backward ramp condition.

In [10]:
posterior_forward = calculate_posterior(prior, 'forward_ramp_condition', probability_table, show_details=True)
posterior_backward = calculate_posterior(prior, 'backward_ramp_condition', probability_table, show_details=True)

Likelihood values p(cube_position|hypothesis):
  p(forward_ramp_condition|H_physics) = 1.0
  p(forward_ramp_condition|H_agent) = 1.0
  p(forward_ramp_condition|H_ramp) = 0.05

  p(forward_ramp_condition|H_physics) * p(H_physics) = 1.0 * 0.9000 = 0.9000
  p(forward_ramp_condition|H_agent) * p(H_agent) = 1.0 * 0.0500 = 0.0500
  p(forward_ramp_condition|H_ramp) * p(H_ramp) = 0.05 * 0.0500 = 0.0025

Σ p(forward_ramp_condition|H_i) * p(H_i) = 0.9525

 p(hypothesis|cube_position):
  p(H_physics|forward_ramp_condition) = 0.9449
  p(H_agent|forward_ramp_condition) = 0.0525
  p(H_ramp|forward_ramp_condition) = 0.0026
Likelihood values p(cube_position|hypothesis):
  p(backward_ramp_condition|H_physics) = 0.05
  p(backward_ramp_condition|H_agent) = 1.0
  p(backward_ramp_condition|H_ramp) = 1.0

  p(backward_ramp_condition|H_physics) * p(H_physics) = 0.05 * 0.9000 = 0.0450
  p(backward_ramp_condition|H_agent) * p(H_agent) = 1.0 * 0.0500 = 0.0500
  p(backward_ramp_condition|H_ramp) * p(H_ramp) = 1.

## 2.5 Weighted Scores
Combines the likelihood-based predictions from each hypothesis into a single score, weighted by how plausible each hypothesis is after seeing the data.

In [11]:
kde_results = pd.read_csv("data/kde_results_demo.csv")
weighted_results =posterior_weighted_score(kde_results, posterior_forward, posterior_backward,show_details=True)

--- FORWARD_RAMP_CONDITION ---
Using posterior probabilities (p(model | condition)):
  physics : 0.9449
  agent   : 0.0525
  ramp    : 0.0026

  TRIAL_A:
    choice_1: physics = 0.9449 × 0.0000000000 = 0.0000000000
    choice_1: agent = 0.0525 × 0.0000000000 = 0.0000000000
    choice_1: ramp = 0.0026 × 0.0000172061 = 0.0000000452
    choice_1 TOTAL: 0.00000004516046367639
    choice_2: physics = 0.9449 × 0.0000000000 = 0.0000000000
    choice_2: agent = 0.0525 × 0.0000000000 = 0.0000000000
    choice_2: ramp = 0.0026 × 0.0000040966 = 0.0000000108
    choice_2 TOTAL: 0.00000001075228174014
    choice_3: physics = 0.9449 × 0.0000025608 = 0.0000024197
    choice_3: agent = 0.0525 × 0.0000025608 = 0.0000001344
    choice_3: ramp = 0.0026 × 0.0000000000 = 0.0000000000
    choice_3 TOTAL: 0.00000255411504013458
    choice_4: physics = 0.9449 × 0.0000099626 = 0.0000094135
    choice_4: agent = 0.0525 × 0.0000099626 = 0.0000005230
    choice_4: ramp = 0.0026 × 0.0000000000 = 0.0000000000
    c

## 2.6  Apply Softmax
Converts weighted scores into choice probabilities.

In [12]:
beta = 200000
softmax_choice_probabilities(weighted_results, beta, show_details=True )

forward_ramp_condition - trial_a
    Choice 1: 0.0920
    Choice 2: 0.0913
    Choice 3: 0.1519
    Choice 4: 0.6648
    Sum: 1.000000

forward_ramp_condition - trial_b
    Choice 1: 0.0884
    Choice 2: 0.0884
    Choice 3: 0.4615
    Choice 4: 0.3617
    Sum: 1.000000

forward_ramp_condition - trial_c
    Choice 1: 0.6289
    Choice 2: 0.1553
    Choice 3: 0.1001
    Choice 4: 0.1157
    Sum: 1.000000

forward_ramp_condition - trial_d
    Choice 1: 0.3560
    Choice 2: 0.4484
    Choice 3: 0.0980
    Choice 4: 0.0976
    Sum: 1.000000

backward_ramp_condition - trial_a
    Choice 1: 0.3381
    Choice 2: 0.1369
    Choice 3: 0.1443
    Choice 4: 0.3807
    Sum: 1.000000

backward_ramp_condition - trial_b
    Choice 1: 0.1601
    Choice 2: 0.1647
    Choice 3: 0.3645
    Choice 4: 0.3106
    Sum: 1.000000

backward_ramp_condition - trial_c
    Choice 1: 0.1196
    Choice 2: 0.0755
    Choice 3: 0.1134
    Choice 4: 0.6915
    Sum: 1.000000

backward_ramp_condition - trial_d
    Choice 

{'forward_ramp_condition': {'trial_a': {'choice_1': np.float64(0.09195397943514225),
   'choice_2': np.float64(0.0913233579269565),
   'choice_3': np.float64(0.15187823946166643),
   'choice_4': np.float64(0.6648444231762348)},
  'trial_b': {'choice_1': np.float64(0.08840318949648397),
   'choice_2': np.float64(0.08842211476269864),
   'choice_3': np.float64(0.46146741756680465),
   'choice_4': np.float64(0.36170727817401277)},
  'trial_c': {'choice_1': np.float64(0.6289025432497105),
   'choice_2': np.float64(0.1552771429963844),
   'choice_3': np.float64(0.1001264148779053),
   'choice_4': np.float64(0.1156938988759998)},
  'trial_d': {'choice_1': np.float64(0.35601233205800187),
   'choice_2': np.float64(0.44841617309236736),
   'choice_3': np.float64(0.09800552921602307),
   'choice_4': np.float64(0.09756596563360767)}},
 'backward_ramp_condition': {'trial_a': {'choice_1': np.float64(0.33806323090626733),
   'choice_2': np.float64(0.1368832412868618),
   'choice_3': np.float64(0.14

# 3 Model Fitting 


## 3.1 Participant data 


In [13]:
# Getting participant data - numer of people for each trial and choice
participant_data = load_participant_data()
print(participant_data)

{'forward_ramp_condition': {'trial_a': {'choice_1': 3, 'choice_2': 2, 'choice_3': 20, 'choice_4': 95}, 'trial_b': {'choice_1': 6, 'choice_2': 3, 'choice_3': 58, 'choice_4': 53}, 'trial_c': {'choice_1': 83, 'choice_2': 23, 'choice_3': 5, 'choice_4': 9}, 'trial_d': {'choice_1': 59, 'choice_2': 50, 'choice_3': 8, 'choice_4': 3}}, 'backward_ramp_condition': {'trial_a': {'choice_1': 50, 'choice_2': 17, 'choice_3': 13, 'choice_4': 39}, 'trial_b': {'choice_1': 21, 'choice_2': 38, 'choice_3': 31, 'choice_4': 29}, 'trial_c': {'choice_1': 11, 'choice_2': 8, 'choice_3': 18, 'choice_4': 82}, 'trial_d': {'choice_1': 12, 'choice_2': 16, 'choice_3': 42, 'choice_4': 49}}}


## 3.2 Parameter Optimization
Finds the best-fitting parameters.


In [14]:
# Run optimization
optimization_result = optimize_parameters(kde_results, participant_data, seed=1)

Running optimization with 10 random restarts (no predefined guesses)
Restart 1/10 (random): p=0.419, q=0.716, beta=1243.737, r=0.302
  Result: p=0.9900, q=0.0100, beta=1243.7436, r=0.0000, log_likelihood=-1321.370878
  New best: -1321.370878
Restart 2/10 (random): p=0.154, q=0.100, beta=1862683.488, r=0.346
  Result: p=0.5775, q=0.2355, beta=1862683.4870, r=0.6599, log_likelihood=-2567.380005
  Log-likelihood: -2567.380005
Restart 3/10 (random): p=0.399, q=0.538, beta=4192003.225, r=0.685
  Result: p=0.5529, q=0.3779, beta=4192003.2241, r=0.5083, log_likelihood=-5365.434272
  Log-likelihood: -5365.434272
Restart 4/10 (random): p=0.210, q=0.871, beta=273973.193, r=0.670
  Result: p=0.8840, q=0.3779, beta=273973.1917, r=0.0525, log_likelihood=-1036.669494
  New best: -1036.669494
Restart 5/10 (random): p=0.419, q=0.558, beta=1403955.347, r=0.198
  Result: p=0.5859, q=0.1958, beta=1403955.3463, r=0.6804, log_likelihood=-2039.581095
  Log-likelihood: -2039.581095
Restart 6/10 (random): p=0

## 3.3 Understanding the Optimization Process

To find the best-fitting parameters, the model maximizes the total log-likelihood of participants’ choices. Specifically, for each option, the log of the model’s predicted probability is multiplied by the number of participants who actually chose that option. The model seeks to maximize the total log-likelihood.

In [15]:
# Detailed log-likelihood analysis
if optimization_result['status'] == 'success':
    p_opt = optimization_result['p_opt']
    q_opt = optimization_result['q_opt']
    beta_opt = optimization_result['beta_opt']
    
    print(f"Using optimized parameters: p={p_opt:.4f}, q={q_opt:.4f}, β={beta_opt:.4f}")
    print()
    
    prior = define_prior(p_opt, q_opt, show_details=False)
    consistency_table = create_probability_table()
    posterior_forward = calculate_posterior(prior, 'forward_ramp_condition', consistency_table, show_details=False)
    posterior_backward = calculate_posterior(prior, 'backward_ramp_condition', consistency_table, show_details=False)
    
    # Get weighted results and probabilities
    weighted_results = posterior_weighted_score(kde_results, posterior_forward, posterior_backward, show_details=False)
    probability_results = softmax_choice_probabilities(weighted_results, beta_opt, show_details=False)
    
    print("Log-likelihood breakdown by condition and trial:")
    print()
    
    total_log_likelihood = 0
    
    for condition in ['forward_ramp_condition', 'backward_ramp_condition']:
        print(f"{condition.upper()}:")
        condition_log_likelihood = 0
        
        for trial in ['trial_a', 'trial_b', 'trial_c', 'trial_d']:
            if condition in probability_results and trial in probability_results[condition]:
                predicted_probs = probability_results[condition][trial]
                choice_counts = participant_data[condition][trial]
                
                trial_log_likelihood = 0
                print(f"  {trial}:")
                
                for choice in ['choice_1', 'choice_2', 'choice_3', 'choice_4']:
                    if choice in predicted_probs and choice in choice_counts:
                        prob = predicted_probs[choice]
                        count = choice_counts[choice]
                        choice_log_likelihood = np.log(prob) * count
                        trial_log_likelihood += choice_log_likelihood
                        
                        print(f"    {choice}: log({prob:.4f}) × {count} = {choice_log_likelihood:.2f}")
                
                print(f"    Trial total: {trial_log_likelihood:.2f}")
                condition_log_likelihood += trial_log_likelihood
        
        print(f"  Condition total: {condition_log_likelihood:.2f}")
        total_log_likelihood += condition_log_likelihood
        print()
    
    print(f"TOTAL LOG-LIKELIHOOD: {total_log_likelihood:.2f}")
    
else:
    print("Cannot perform detailed analysis - optimization failed")


Using optimized parameters: p=0.8840, q=0.3779, β=273973.1917

Log-likelihood breakdown by condition and trial:

FORWARD_RAMP_CONDITION:
  trial_a:
    choice_1: log(0.0517) × 3 = -8.89
    choice_2: log(0.0517) × 2 = -5.92
    choice_3: log(0.1043) × 20 = -45.21
    choice_4: log(0.7923) × 95 = -22.12
    Trial total: -82.14
  trial_b:
    choice_1: log(0.0536) × 6 = -17.55
    choice_2: log(0.0536) × 3 = -8.78
    choice_3: log(0.5203) × 58 = -37.89
    choice_4: log(0.3724) × 53 = -52.35
    Trial total: -116.57
  trial_c:
    choice_1: log(0.7599) × 83 = -22.78
    choice_2: log(0.1101) × 23 = -50.75
    choice_3: log(0.0595) × 5 = -14.11
    choice_4: log(0.0705) × 9 = -23.87
    Trial total: -111.52
  trial_d:
    choice_1: log(0.3693) × 59 = -58.78
    choice_2: log(0.5079) × 50 = -33.87
    choice_3: log(0.0616) × 8 = -22.30
    choice_4: log(0.0612) × 3 = -8.38
    Trial total: -123.33
  Condition total: -433.56

BACKWARD_RAMP_CONDITION:
  trial_a:
    choice_1: log(0.7542) × 

## 3.4 Model Predictions vs. Participant Results
Shows the model predictions and participants' choices.


In [16]:
comparison_data = []
for condition in ['forward_ramp_condition', 'backward_ramp_condition']:
    condition_display = 'Forward' if 'forward' in condition else 'Backward'
    for trial in ['trial_a', 'trial_b', 'trial_c', 'trial_d']:
        if condition in probability_results and trial in probability_results[condition]:
            predicted_probs = probability_results[condition][trial]
            choice_counts = participant_data[condition][trial]
                
            # Calculate human proportions
            total_count = sum(choice_counts.values())
            human_props = [choice_counts.get(f'choice_{i+1}', 0) / total_count for i in range(4)]
                
            comparison_data.append({
                'Trial': trial.replace('trial_', ''),
                'Condition': condition_display,
                'Target_Percentage_1': human_props[0],
                'Target_Percentage_2': human_props[1], 
                'Target_Percentage_3': human_props[2],
                'Target_Percentage_4': human_props[3],
                'Predicted_Percentage_1': predicted_probs.get('choice_1', 0),
                'Predicted_Percentage_2': predicted_probs.get('choice_2', 0),
                'Predicted_Percentage3': predicted_probs.get('choice_3', 0),
                'Predicted_Percentage4': predicted_probs.get('choice_4', 0)
            })
    
model_comparison = pd.DataFrame(comparison_data)
display(model_comparison.round(4))

,Trial,Condition,Target_Percentage_1,Target_Percentage_2,Target_Percentage_3,Target_Percentage_4,Predicted_Percentage_1,Predicted_Percentage_2,Predicted_Percentage3,Predicted_Percentage4
0,a,Forward,0.0250,0.0167,0.1667,0.7917,0.0517,0.0517,0.1043,0.7923
1,b,Forward,0.0500,0.0250,0.4833,0.4417,0.0536,0.0536,0.5203,0.3724
2,c,Forward,0.6917,0.1917,0.0417,0.0750,0.7599,0.1101,0.0595,0.0705
3,d,Forward,0.4917,0.4167,0.0667,0.0250,0.3693,0.5079,0.0616,0.0612
4,a,Backward,0.4202,0.1429,0.1092,0.3277,0.7542,0.0807,0.0524,0.1127
5,b,Backward,0.1765,0.3193,0.2605,0.2437,0.2282,0.2446,0.2803,0.2470
6,c,Backward,0.0924,0.0672,0.1513,0.6891,0.0086,0.0086,0.0264,0.9565
7,d,Backward,0.1008,0.1345,0.3529,0.4118,0.1242,0.1242,0.3968,0.3548
